# ReAct basics

https://react-lm.github.io/

https://arxiv.org/pdf/2210.03629

## Setup LLM

In [1]:
from dotenv import load_dotenv
import sys
import json

from langchain.prompts import PromptTemplate

# Load the file that contains the API keys - OPENAI_API_KEY
load_dotenv('C:\\Users\\raj\\.jupyter\\.env')

# setting path
sys.path.append('../')

from utils.create_chat_llm import create_gpt_chat_llm, create_cohere_chat_llm, create_anthropic_chat_llm
from utils.create_llm import create_cohere_llm, create_gpt_llm

# Try with GPT
llm = create_gpt_chat_llm({"temperature":0.1}) #, "model_kwargs":{"top_p":0.1}})

## Select a question from HotPotQA

https://hotpotqa.github.io/explorer.html

In [2]:
question = " Are both Superdrag and Collective Soul rock bands?"

## 1. Try with Chain of Thought (CoT)

Doesn't interact with the environment. Responses are solely based on LLM's parametric knowledge.

In [3]:

question_str= f"""
Answer the question:

{question}

Think step by step.
"""

response = llm.invoke(question_str.format(question))

print(response.content)

1. Superdrag is a rock band.
2. Collective Soul is also a rock band.
3. Therefore, both Superdrag and Collective Soul are rock bands.


## 2. Try out tool calling agent

https://python.langchain.com/v0.1/docs/integrations/tools/wikipedia/


In [4]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent, create_react_agent
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

system_message = """
You are a polite agent who uses tools to answer questions.

Keep the answer concise.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("human", "answer the question: {question}"),
    ("placeholder", "{agent_scratchpad}"),
])


## 3. ReAct agent

https://api.python.langchain.com/en/latest/agents/langchain.agents.react.agent.create_react_agent.html

Prompt source: https://smith.langchain.com/hub/hwchase17/react

In [30]:
question = "SR connector is based on the connector that is often used for what type of cable?"

prompt_template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action

... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer

Final Answer: the final answer to the original input question

Here are some examples:
{examples}

Begin!

Question: {question}

Thought:{agent_scratchpad}
"""

prompt = PromptTemplate(
    template = prompt_template,
    input_variables = ["tools", "question", "examples", "agent_scratchpad"]
)

In [49]:
examples = """

Question: who is older Einstien or Newton
Thought:I need to find out the birth dates of both Einstein and Newton to determine who is older.
Action: wikipedia[Isaac Newton]
Action Input: Isaac Newton
Observation: Isaac Newton was born on December 25, 1642
Action: wikipedia[Albert Einstien]
Action Input: Albert Einstien
Observation: Albert Einstein was born on March 14, 1879
Thought: I now know the final answer to the original question.
Action: Finish (Newton is older)
"""

examples = """

Question: who is older Einstien or Newton
Thought:I need to find out the birth dates of both Einstein and Newton to determine who is older.
Action: search[Isaac Newton]
Observation: Isaac Newton was born on December 25, 1642
Action: search[Albert Einstien]
Observation: Albert Einstein was born on March 14, 1879
Thought: I now know the final answer to the original question.
Action: Finish (Newton is older)
"""

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, verbose=True)
response = agent_executor.invoke({"question": question, "examples": examples})



> Entering new AgentExecutor chain...
I need to find out what type of cable the SR connector is often used for.

Action: wikipedia
Action Input: SR connectorPage: SR connector
Summary: The SR connector or CP connector (Russian: Соединитель Радиочастотный, 'radio frequency connector') is a Russian-made radio frequency connector for coaxial cables. It is based on the American BNC connector. Types with an American equivalent have slightly different dimensions due to discrepancies in imperial to metric conversion, though with some force the two can still be mated. There are other types which do not resemble their American counterpart.
Most SR connectors are variants of SR-50 (50 Ω) or SR-75 (75 Ω) versions, with the SR-75 typically having a thinner center pin. They often resemble C connectors in shape, and have threaded inserts similar to N connectors. Further numerical suffixes denote specific kinds of connectors, for instance the CP 75-164 is a much larger high power connector, designe

In [50]:
response

{'question': 'SR connector is based on the connector that is often used for what type of cable?',
 'examples': '\n\nQuestion: who is older Einstien or Newton\nThought:I need to find out the birth dates of both Einstein and Newton to determine who is older.\nAction: search[Isaac Newton]\nObservation: Isaac Newton was born on December 25, 1642\nAction: search[Albert Einstien]\nObservation: Albert Einstein was born on March 14, 1879\nThought: I now know the final answer to the original question.\nAction: Finish (Newton is older)\n',
 'output': 'The SR connector is based on the BNC connector, which is often used for coaxial cables.'}

In [47]:
print(response["intermediate_steps"])

[(AgentAction(tool='wikipedia', tool_input='SR connector', log='I need to find out what type of cable the SR connector is often used for.\n\nAction: wikipedia\nAction Input: SR connector'), 'Page: SR connector\nSummary: The SR connector or CP connector (Russian: Соединитель Радиочастотный, \'radio frequency connector\') is a Russian-made radio frequency connector for coaxial cables. It is based on the American BNC connector. Types with an American equivalent have slightly different dimensions due to discrepancies in imperial to metric conversion, though with some force the two can still be mated. There are other types which do not resemble their American counterpart.\nMost SR connectors are variants of SR-50 (50 Ω) or SR-75 (75 Ω) versions, with the SR-75 typically having a thinner center pin. They often resemble C connectors in shape, and have threaded inserts similar to N connectors. Further numerical suffixes denote specific kinds of connectors, for instance the CP 75-164 is a much 